In [1]:
import argparse
import os
import time

import chainer
import chainer.links as L
import chainer.functions as F
import cv2
import numpy as np

from networks.mobilenetv2 import MobilenetV2
from networks.vgg16 import VGG16
from networks.resnet50 import ResNet50
from predict import prepare_setting

In [2]:
from collections import namedtuple

Args = namedtuple("Args",("dataset", "model_path", "device"))
args  = Args(dataset="/home/terasaki/dataset/food-101/",
             model_path="resnet50/",
             device=-1)

classes = np.genfromtxt(os.path.join(args.dataset, "meta", "classes.txt"),
                            str,
                            delimiter="\n")


In [3]:
model, preprocess, xp, test_dataset = prepare_setting(args)

resnet50/model_epoch_24.npz
resnet50/args.json


In [4]:
from matplotlib import pyplot as plt
import random
%matplotlib inline

from ipywidgets import interact

In [5]:
sample_indices=random.sample(range(len(test_dataset)),10)

In [6]:
def predict_sample(idx):
    img, label= test_dataset.base[idx]
    input_image = test_dataset[idx][0]
    with chainer.using_config('train', False):
        h = model.predictor(xp.expand_dims(xp.array(input_image), axis=0))
    prediction = chainer.functions.softmax(h)
    
    if args.device >= 0:
        prediction = xp.asnumpy(prediction[0].data)
    else:
        prediction = prediction[0].data
    top_ten = np.argsort(-prediction)[:10]
    print(classes[top_ten])
    if classes[label] != classes[top_ten][0]:
        plt.title("{} {}".format(classes[label], classes[top_ten][0]))
    else:
        plt.title("{} {}".format(classes[label], "bingo"))
    plt.imshow(img.transpose(1,2,0).astype(np.uint8))
    
interact(predict_sample, idx=sample_indices)

interactive(children=(Dropdown(description='idx', options=(17397, 4959, 17683, 3796, 16979, 3580, 24500, 7712,…

<function __main__.predict_sample(idx)>